## RETRIEVE ADDITIONAL DATA FROM KNOW YOUR MEMES WEBSITE: 

- AUTHOR, 
- LAST UPDATER, 
- ADDED DATE, 
- LAST UPDATE DATE, 
- NUMBER OF VIEWS

In [17]:
import numpy as np
import pandas as pd
import json 
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup # pip install beautifulsoup4
import nltk
import re
from datetime import date
import datetime
import time

path_to_cleaned = "cleaned_data//"
fname_main_df = 'main_df.csv'
df_main = pd.read_csv(path_to_cleaned + fname_main_df, sep=';', index_col=0, encoding='utf-8') 

# Web crawling rules: https://knowyourmeme.com/robots.txt

In [12]:
your_email = ' ' # for bot header  
result_file = "web_scraping.txt" # results will be gathered here

start_idx = 11964
end_idx = 12196 # 12196  # len(ids) # 8491/12196

In [3]:
def retrieve_data(i, ids, headers):
    url = ids[i]
    response = requests.get(url, headers=headers)
    status = response.status_code
    if status==200:
        content = response.text
    else: 
        content = None    
        
    return [status, content]

# Status codes:  200 - ok, 401- no access rights, 404- not found, 500- server broke 

In [4]:
def parse_html(content):
    
    soup = BeautifulSoup(content, 'html.parser')
    
    # Views: 
    c =  soup.find_all("div", class_ = "views")[0] 
    c.find_all("div", "title")
    c.find_all(re.compile('title'))
    views = c.get('title')  # '136,011 Views'
    views = views.split(' ')[0] 
    views = re.sub(',', '', views) # 136011
       
    # Added_by, updated_by, add_date, update_date: 
    b = soup.find_all("div", class_ = "author-info")[0]
    p_list = b.find_all("p")
    added_by, updated_by, added_date, update_date = "", "", "", ""
    
    for p in p_list:
        if "Added" in p.get_text().strip():
            a = p.find_all("a")[0]
            added_by = a.attrs["href"].split('users/')[-1]
            t = p.find_all("abbr")[0]
            added_date = t.attrs["title"].split('T')[0]
            
        elif "Updated" in p.get_text().strip():
            a = p.find_all("a")[0]
            updated_by = a.attrs["href"].split('users/')[-1]
            t = p.find_all("abbr")[0]
            update_date = t.attrs["title"].split('T')[0]
    
    return [views, added_by, updated_by, added_date, update_date]

In [5]:
# Bot: 
bot_header = "univ. course project"  # "dataeng (" + your_email + ") / student project"
headers = requests.utils.default_headers()
headers.update(
  { 'User-Agent': bot_header, }
)

# Links:
#id_df = pd.read_excel('id_df.xlsx', index_col=0)  
#ids = id_df['url'].tolist()
df_main['url'].tolist()

In [20]:
# Scraping:
for i in range(start_idx, end_idx):  # len(ids) # 12196
    status, content = retrieve_data(i, ids, headers)
    if status!=200: # some error occurred
        result = [str(i), ids[i], str(status), "0", "", "", "", ""] # to record bad links
    else:
        result = [str(i), ids[i], str(status)] + parse_html(content)
        
    row = ",".join(result) # for writing list to txt file
        
    # write to file:
    with open(result_file, "a", encoding="UTF-8") as f:
        f.write(row)
        f.write("\n")
    
    # Report everything is going fine after 50 requests (ca 8.3 min):
    if i%50==0:
        print(i)   
    time.sleep(30) # delay